## Modeling RNN by using time-series data

In [5]:
# import library
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [6]:
# number of nodes of each neurons
n_hidden = 35
lr = 0.01
epochs = 1000

string = 'hello pytorch. how long can a rnn cell remember?'
chars = 'abcedfghijklmnopqrstuvwxyz ?!.,:;01'
char_list = [i for i in chars]
n_letters = len(char_list)

In [7]:
# convert string to one-hot vector
def string_to_onehot(string):
    start = np.zeros(shape = len(char_list), dtype = int)
    end = np.zeros(shape = len(char_list), dtype = int)
    start[-2] = 1
    end [-1] = 1
    
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape = n_letters, dtype = int)
        zero[idx] = 1
        start = np.vstack([start, zero])
    output = np.vstack([start, end])
    return output

In [8]:
# convert one-hot vector into string
def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

In [9]:
# Recurrent Neural Network
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, output_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.act_fn = nn.Tanh()
        
    def forward(self, input, hidden):
        hidden = self.act_fn(self.i2h(input)+self.h2h(hidden))
        output = self.i2o(hidden)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
    
rnn = RNN(n_letters, n_hidden, n_letters)

# define loss function and optimization function
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr = lr)

# training
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    rnn.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()
    
    for j in range(one_hot.size()[0]-1):
        input_ = one_hot[j:j+1, :]
        target = one_hot[j+1]
        
        output, hidden = rnn.forward(input_, hidden)
        loss = loss_func(output.view(-1), target.view(-1))
        total_loss += loss
        
    total_loss.backward()
    optimizer.step()
    
    if i%10 == 0:
        print(total_loss)

tensor(2.5712, grad_fn=<AddBackward0>)
tensor(0.9538, grad_fn=<AddBackward0>)
tensor(0.6199, grad_fn=<AddBackward0>)
tensor(0.4200, grad_fn=<AddBackward0>)
tensor(0.2871, grad_fn=<AddBackward0>)
tensor(0.2167, grad_fn=<AddBackward0>)
tensor(0.1608, grad_fn=<AddBackward0>)
tensor(0.1299, grad_fn=<AddBackward0>)
tensor(0.1027, grad_fn=<AddBackward0>)
tensor(0.1030, grad_fn=<AddBackward0>)
tensor(0.0777, grad_fn=<AddBackward0>)
tensor(0.0661, grad_fn=<AddBackward0>)
tensor(0.0762, grad_fn=<AddBackward0>)
tensor(0.0563, grad_fn=<AddBackward0>)
tensor(0.0505, grad_fn=<AddBackward0>)
tensor(0.0528, grad_fn=<AddBackward0>)
tensor(0.0475, grad_fn=<AddBackward0>)
tensor(0.0416, grad_fn=<AddBackward0>)
tensor(0.0562, grad_fn=<AddBackward0>)
tensor(0.0371, grad_fn=<AddBackward0>)
tensor(0.0327, grad_fn=<AddBackward0>)
tensor(0.0302, grad_fn=<AddBackward0>)
tensor(0.0330, grad_fn=<AddBackward0>)
tensor(0.0298, grad_fn=<AddBackward0>)
tensor(0.0249, grad_fn=<AddBackward0>)
tensor(0.0231, grad_fn=<A

In [10]:
# validate whether model trained or not
start = torch.zeros(1, len(char_list))
start[:, -2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()  # initialize output of the hidden vector
    input_ = start
    output_string = ""
    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        output_string += onehot_to_word(output.data)
        input_ = output
        
    print(output_string)

hello pelorwel? welog elmeebepwelwemlmellmeelmem


## Embedding + RNN, LSTM, GRU

In [11]:
import os 

try:
  os.mkdir("./data")
except:
  pass

!wget https://raw.githubusercontent.com/GunhoChoi/PyTorch-FastCampus/master/05_RNN/2_Char_RNN/data/linux.txt -P ./data

--2020-12-07 21:08:34--  https://raw.githubusercontent.com/GunhoChoi/PyTorch-FastCampus/master/05_RNN/2_Char_RNN/data/linux.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.76.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.76.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33756 (33K) [text/plain]
Saving to: ‘./data/linux.txt.1’

linux.txt.1         100%[===================>]  32.96K  --.-KB/s    in 0.03s   

2020-12-07 21:08:35 (1.18 MB/s) - ‘./data/linux.txt.1’ saved [33756/33756]



In [12]:
import torch
import torch.nn as nn
import unidecode
import string
import random
import re
import time, math

In [13]:
num_epochs = 2000
print_every = 100
plot_every = 10
chunk_len = 200
hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

In [14]:
# Commonly used part
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


In [15]:
# import data
file = unidecode.unidecode(open('./data/linux.txt').read())
file_len = len(file)
print('file_len = ', file_len)

file_len =  33756


In [16]:
# Since all files can't be read at once, we need to divide it in certain length.
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index : end_index]

# convert string list into index
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor("ABCdef"))

# split indexed string into input and target
def random_training_set():
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

tensor([36, 37, 38, 13, 14, 15])


In [36]:
# class of whole process(embedding + RNN + decoding)
# First RNN
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers = 1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        out = self.encoder(input.view(1, -1))
        out, hidden = self.rnn(out, hidden)
        out = self.decoder(out.view(batch_size, -1))
        return out, hidden
    
    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, hidden_size)
        return hidden        
    

model = RNN(input_size = n_characters, embedding_size = embedding_size, 
            hidden_size = hidden_size, output_size = n_characters, num_layers = 2)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

# testing code
inp = char_tensor('A')
hidden = model.init_hidden()
out, hidden = model(inp, hidden)

def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden = model(x,hidden)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)


for i in range(num_epochs):
    inp,label = random_training_set()
    hidden = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden = model(x,hidden)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()


 tensor([4.6298], grad_fn=<DivBackward0>) 

ns.!nc;"v:`=K 3d^8W)(AB;dp*
lSu?>*jt]i!3i9\;O
rx_su^nB5##a{lp~TX}tIkoza<4/[876b}ok:Ii7d3#!SDiqyVyOi80pv~@HcP}CI8l??-s'=*h
 tensor([2.5853], grad_fn=<DivBackward0>) 

ber.
beU erdan sat loue iil] thread on areele
By saes noris os and sine so o cis;
Thenite to bans phald faive sawet thite het ine wen.:

Th ye he sacos foret te hareantt ine anin
Afs ;oud be lein yalh 
 tensor([2.1391], grad_fn=<DivBackward0>) 

bomertertm be wid,
 meand the tuere brerorcell,
Do I and my go ther! lort, to.
kuch our my Marte?

Uun mobe hit ard you'd mesos he hee the hitel


CIANWANRRSRIS:
C tham! brovent chom yene mowr th
 tensor([2.2910], grad_fn=<DivBackward0>) 

bert
Howind of a ifeide of to the lor taresou lare-

LUCELOANTE:
Ans my arecs the deve of fice mingj nut home wat
The gust dor thid wak is ir nor word'd be strour st the mat hatipe sime how me the mant
 tensor([2.1721], grad_fn=<DivBackward0>) 

bros: besther, secliargbe wards me to for with these get

In [30]:
# Second LSTM
import os 

try:
  os.mkdir("./data")
except:
  pass

!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt -P ./data

--2020-12-07 21:47:23--  https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.88.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.88.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘./data/input.txt’

input.txt           100%[===================>]   1.06M  41.0KB/s    in 16s     

2020-12-07 21:47:40 (66.2 KB/s) - ‘./data/input.txt’ saved [1115394/1115394]



In [31]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

file = unidecode.unidecode(open('./data/input.txt').read())
file_len = len(file)
print('file_len =', file_len)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100
file_len = 1115394


In [38]:
# Second LSTM
class LSTM(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers = 1):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden, cell):
        out = self.encoder(input.view(1, -1))
        out, (hidden, cell) = self.lstm(out, (hidden, cell))
        out = self.decoder(out.view(batch_size, -1))
        return out, hidden, cell
    
    def init_hidden(self):
        hidden = torch.zeros(num_layers, batch_size, hidden_size)
        cell = torch.zeros(num_layers, batch_size, hidden_size)
        return hidden, cell
    

model = LSTM(input_size = n_characters, embedding_size = embedding_size, 
            hidden_size = hidden_size, output_size = n_characters, num_layers = 1)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

# testing code
inp = char_tensor("A")
hidden,cell = model.init_hidden()
out,hidden,cell = model(inp,hidden,cell)

def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden, cell = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden, cell = model(x, hidden, cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)


for i in range(num_epochs):
    inp,label = random_training_set()
    hidden, cell = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden, cell = model(x,hidden, cell)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()    


 tensor([4.5834], grad_fn=<DivBackward0>) 

W			fLTTP$/*QUt]%Uh~Pld
czVpN*|5mX%ho*%'zZzX'O %$SsWwV^
lGUKS-99W
 tensor([4.5834], grad_fn=<DivBackward0>) 

W			fLTTP$/*QUt]%Uh~Pld
czVpN*|5mX%ho*%'zZzX'O %$SsWwV^
Wxy)XCr\^yz&IYA	p_m*2;k'l<%
Pk>v%wVnz1Yu#_(sY	L8s*
 tensor([2.5464], grad_fn=<DivBackward0>) 

bDC(qhyy bert, yeyv
Tpu wouknyi helam, sut y amiere
Thot h elekct ke.ist, hanene nad besenads ounyoy soue
et mi
-rer gtok

ETITatn thisMeurr t ft soasouleA moes yovnl hery. hire woramimn mecele wthe sp
 tensor([2.2679], grad_fn=<DivBackward0>) 

b"yst are madt,
:aindeplser the it thow alist, cond be o sou bouch dog, the pour thintrigand youse ovonle that indco. gemaves tha won bad thet boy thes ther ncot the sin cod rot
cot hou thig the, min g
 tensor([2.4304], grad_fn=<DivBackward0>) 

bLTht the toF hela?

MAAAS:
Gpis od tey andeslel tie hit oh dony liand reit his sat?
O thapre thee not four that ho thes no we dester and the dlace powre thes to hy sat. lousat thitht marrsesler o

In [39]:
# First RNN
class GRU(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers = 1):
        super(GRU, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.gru = nn.GRU(embedding_size, hidden_size, num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        out = self.encoder(input.view(1, -1))
        out, hidden = self.gru(out, hidden)
        out = self.decoder(out.view(batch_size, -1))
        return out, hidden
    
    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, hidden_size)
        return hidden        
    

model = GRU(input_size = n_characters, embedding_size = embedding_size, 
            hidden_size = hidden_size, output_size = n_characters, num_layers = 1)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

# testing code
inp = char_tensor('A')
hidden = model.init_hidden()
out, hidden = model(inp, hidden)

def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden = model(x,hidden)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)


for i in range(num_epochs):
    inp,label = random_training_set()
    hidden = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden = model(x,hidden)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()


 tensor([4.5806], grad_fn=<DivBackward0>) 

bR )~qq866^})3k2/}c RC]Z@7%ld	i]z]oOEa]dA:H#eutQiON`TiW?5a<R	nqY"IwW/)8?I[t)[KW($HQbH &*+Ub~9.45od7UW\C]z-W"z}nv}SE0e\Jauh;}W[(5ljjb!0MLV"xE/IS0
[V6b2IzdbSq1uN<+b
YdH:Kbs,SI|Gat+m4"8+$-qgE{{5\l/Y,p 
 tensor([2.4970], grad_fn=<DivBackward0>) 

bol me ling il sat Lhatf the,
RAU)XJ_kAnged posan .at the foure man lo;e outh band arery pave, witrd in it co' gomif hime do, s Vithe the fill ,erce sa wesat  aay satlire pot bilov bil, Cins aor lous! 
 tensor([2.5070], grad_fn=<DivBackward0>) 

blow ha:
Bhater:

GUCgzlly eason so'd as wir, hase, Rreess me mord ooptamer you nothe your chilr ay, ailll he loratrat sne shers, the for sathon, nour drise be in it dolld sece, finse thim the sy erdes
 tensor([2.2137], grad_fn=<DivBackward0>) 

bunt cofe if thet fousen the, an fardis wis her of, wall ar fut cisinges hesine,
Waich sot word word,
Ter of the and at nis ther, I hate wary bis ing to his weer thest so.

CKNEO: Ga ceperit in sing we
 tensor([2.2038